In [ ]:
! pip install kaggle wget

In [ ]:
! nvidia-smi

In [ ]:
! mkdir ~ /.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets download -d gverzea/edible-wild-plants
! unzip /content/edible-wild-plants.zip

In [ ]:
!git clone https://github.com/alexfrst/data-augmentation-benchmark

In [3]:
import sys

import torch
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

sys.path.append("data-augmentation-benchmark")

import model.model as model
from model.evaluate import evaluate
from model.train import train_model
from utils.dataset import load_dataset
from utils.print_utils import Symbols


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

print(f"Working on {device} {Symbols.OK}")

train_loader, val_loader, test_loader, nb_classes = load_dataset("datasets/dataset", "datasets/dataset-test", batch_size=85)
inception_v3 = model.load_inception_for_tl(nb_classes, device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(inception_v3.AuxLogits.fc.parameters(), lr=0.001, momentum=0.9)

print("Apprentissage en transfer learning")
inception_v3.to(device)
inception_v3.train(True)
torch.manual_seed(42)

train_model(inception_v3, train_loader, val_loader, optimizer, criterion, evaluate, 65, device=device)
inception_v3.train(False)
loss, accuracy = evaluate(inception_v3, test_loader, device, criterion)
print("Accuracy (test): %.1f%%" % (100 * accuracy))

Working on cuda:0 ✓
Loading dataset...
Nombre d'images de train : 5902
Nombre d'images de val : 656
Nombre d'images de test : 310
Apprentissage sur 62 classes
Data loading and integrity check done ✓
Apprentissage en transfer learning


Epoch 0: 3it [01:22, 25.47s/it, Acc (val)=0.317, loss train=4.15, loss val=4.16]Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x000001F6CECC6A70>
Traceback (most recent call last):
  File "C:\Users\alexf\anaconda3\envs\AdIA\lib\site-packages\torch\utils\data\dataloader.py", line 1358, in __del__
    self._shutdown_workers()
  File "C:\Users\alexf\anaconda3\envs\AdIA\lib\site-packages\torch\utils\data\dataloader.py", line 1322, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "C:\Users\alexf\anaconda3\envs\AdIA\lib\multiprocessing\process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "C:\Users\alexf\anaconda3\envs\AdIA\lib\multiprocessing\popen_spawn_win32.py", line 108, in wait
    res = _winapi.WaitForSingleObject(int(self._handle), msecs)
KeyboardInterrupt: 
Epoch 0: 3it [01:35, 31.98s/it, Acc (val)=0.317, loss train=4.15, loss val=4.16]

KeyboardInterrupt

